In [1]:
from confluent_kafka import Producer
from confluent_kafka import Consumer
import ccloud_lib
from faker import Faker
import time
import json

In [2]:
TOPIC = "faker-users"
GROUP_ID = "faker-groups"

In [3]:
# Create Producer instance
conf = ccloud_lib.read_ccloud_config("python.config")
producer_conf = ccloud_lib.pop_schema_registry_params_from_config(conf)
producer = Producer(producer_conf)

%4|1658854764.657|CONFWARN|rdkafka#producer-1| [thrd:app]: Configuration property session.timeout.ms is a consumer property and will be ignored by this producer instance


In [6]:
producer

In [21]:
# Create topic if needed
ccloud_lib.create_topic(conf, TOPIC)

%4|1658037343.531|CONFWARN|rdkafka#producer-5| [thrd:app]: Configuration property session.timeout.ms is a consumer property and will be ignored by this producer instance


In [23]:
fake=Faker()

In [24]:
for i in range(10):
    data={"name":fake.name(),
          "age":fake.random_int(min=18, max=80, step=1),
          "street":fake.street_address(),
          "city":fake.city(),
          "state":fake.state(),
          "zip":fake.zipcode()}
    m=json.dumps(data)
    producer.poll(0)
    producer.produce(TOPIC, m.encode('utf-8'), callback=ccloud_lib.acked)

In [25]:
producer.flush()

Produced record to topic faker-users partition [0] @ offset 20
Produced record to topic faker-users partition [0] @ offset 21
Produced record to topic faker-users partition [0] @ offset 22
Produced record to topic faker-users partition [0] @ offset 23
Produced record to topic faker-users partition [0] @ offset 24
Produced record to topic faker-users partition [0] @ offset 25
Produced record to topic faker-users partition [0] @ offset 26
Produced record to topic faker-users partition [0] @ offset 27
Produced record to topic faker-users partition [0] @ offset 28
Produced record to topic faker-users partition [0] @ offset 29


0

In [26]:
# Create Consumer instance
conf = ccloud_lib.read_ccloud_config("python.config")
consumer_conf = ccloud_lib.pop_schema_registry_params_from_config(conf)
consumer_conf['group.id'] = GROUP_ID
consumer_conf['auto.offset.reset'] = 'earliest'
consumer = Consumer(consumer_conf)

In [27]:
topics = consumer.list_topics()
topics.topics[TOPIC].partitions

{0: PartitionMetadata(0)}

In [28]:
# Subscribe to topic
consumer.subscribe([TOPIC])

In [29]:
while True:
    msg = consumer.poll(1.0) #timeout
    if msg is None:
        print("Waiting for message or event/error in poll()")
        continue
    elif msg.error():
        print("Error: {}".format(msg.error()))
        continue
    else:
        data=msg.value().decode('utf-8')
        print(data)

Waiting for message or event/error in poll()
Waiting for message or event/error in poll()
Waiting for message or event/error in poll()
Waiting for message or event/error in poll()
Waiting for message or event/error in poll()
Waiting for message or event/error in poll()
Waiting for message or event/error in poll()
Waiting for message or event/error in poll()
Waiting for message or event/error in poll()
Waiting for message or event/error in poll()
Waiting for message or event/error in poll()
Waiting for message or event/error in poll()
{"name": "Brittany Goodwin", "age": 66, "street": "5658 Parrish Curve Suite 551", "city": "Keithview", "state": "Virginia", "zip": "82909"}
{"name": "Anthony Proctor", "age": 40, "street": "613 Novak Harbor Apt. 661", "city": "Port Tammyhaven", "state": "Delaware", "zip": "19099"}
{"name": "Matthew Hale", "age": 67, "street": "64473 Bowers Ridge", "city": "Port Jamesshire", "state": "Wisconsin", "zip": "84143"}
{"name": "Stephanie Blackburn", "age": 29, "s

KeyboardInterrupt: 